In [1]:
import torch
import torchvision.models as models

In [2]:
from bestfitting.protein_clean.src.net import _init_paths
from bestfitting.protein_clean.src.net import densenet
from bestfitting.protein_clean.src.config import config

run on Wooseoks-MacBook-Pro.local
run on Wooseoks-MacBook-Pro.local


In [21]:
import os
opj = os.path.join

In [24]:
pretrained_file = opj(config.PRETRAINED_DIR, densenet.model_names['class_densenet121_large_dropout'])
net = densenet.class_densenet121_large_dropout(num_classes=28, in_channels=4, pretrained_file=pretrained_file)

In [40]:
weights = opj(config.RESULT_DIR, 'models/external_crop1024_focal_slov_hardlog_clean_class_densenet121_large_dropout_i1536_aug2_5folds/fold0/final.pth')
net.load_state_dict(torch.load(weights)['state_dict'])

<All keys matched successfully>